### Notebook to produce the dataset with 30years already normalized and reduced(pca) with the results obtained from the analysis over the 30y_even

OUTPUT : matrix with 30y (1994-2023) normalized and with pca applied
- X_pca_30y.npy

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [2]:
import xarray as xr
import numpy as np
import gc
import joblib
import warnings
warnings.filterwarnings('ignore')

import importlib
import utils.data_processing
import utils.visualization
importlib.reload(utils.data_processing)
importlib.reload(utils.visualization)

# Import the functions 
from utils.data_processing import prepare_data_matrix, apply_global_standardization, perform_incremental_pca



In [3]:
# Return to the original dataset to process the 2009-2023 data
gc.collect()
ds = xr.open_dataset('../era5_1994_2008.grib', engine='cfgrib')

# 
z_500 = ds['z'].sel(isobaricInhPa=[500])
t_850 = ds['t'].sel(isobaricInhPa=[850])
z_500 = z_500.rename({'isobaricInhPa': 'pressure_z'})
t_850 = t_850.rename({'isobaricInhPa': 'pressure_t'})
ds_filtered = xr.Dataset({'z': z_500, 't': t_850})

del ds, z_500, t_850
gc.collect()

Ignoring index file '../era5_1994_2008.grib.5b7b6.idx' incompatible with GRIB file


0

In [4]:
X, _ = prepare_data_matrix(ds_filtered)
global_mean = np.load("global_mean_30y.npy")
global_std  = np.load("global_std_30y.npy")

X_std = (X - global_mean) / global_std

del ds_filtered, X
gc.collect()



Processing z...
     → z: ('time', 'pressure_z', 'latitude', 'longitude') → (5479, 64521)
Processing t...
     → t: ('time', 'pressure_t', 'latitude', 'longitude') → (5479, 64521)

Combined matrix shape: (5479, 129042)


22

In [5]:
# Load the PCA model
ipca = joblib.load("ipca_30y.pkl")
X_reduced = ipca.transform(X_std)

# 
np.save("X_pca_1994_2008.npy", X_reduced)
print("Dati ridotti salvati in X_pca_1994_2008.npy")

Dati ridotti salvati in X_pca_1994_2008.npy


The previous code as been applied to the two files: era5_1994_2008.grib and era5_2009_2023.grib. Now I have obtained the two matrices X_pca_...period..., and now I merge them in order to obtain the final 30 years matrix with the normalization and pca applied

#### 30 years matrix

In [ ]:

X1 = np.load("X_pca_1994_2008.npy")
X2 = np.load("X_pca_2009_2023.npy")

X_pca_30y = np.concatenate([X1, X2], axis=0)

print(f"Shape unified matrix: {X_pca_30y.shape}")

# 
np.save("X_pca_30y.npy", X_pca_30y)


Shape unified matrix: (10957, 20)
